<a href="https://colab.research.google.com/github/trduc97/Spotify_classification/blob/main/Spotify_enriching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import libraries and datafiles

In [ ]:
import gdown
import pandas as pd

# Define the URL of the CSV file on Google Drive
train_url = 'https://drive.google.com/uc?id=1xta0FaTHYVKjLbc7ztPgDME5nsqZGoPz'
test_url = 'https://drive.google.com/uc?id=1O0DtiXCsgl7CDV52yOvVWwaKriXjqWxL'

# Define the output file name
train_file = 'classification_train.csv'
test_file = 'classification_test.csv'
# Download the file from Google Drive
gdown.download(train_url, train_file, quiet=False)
gdown.download(test_url, test_file, quiet=False)

# Read the CSV file into a DataFrame
train_set = pd.read_csv(train_file)
test_set = pd.read_csv(test_file)



Downloading...
From: https://drive.google.com/uc?id=1xta0FaTHYVKjLbc7ztPgDME5nsqZGoPz
To: /content/classification_train.csv
100%|██████████| 39.1k/39.1k [00:00<00:00, 19.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0DtiXCsgl7CDV52yOvVWwaKriXjqWxL
To: /content/classification_test.csv
100%|██████████| 8.57k/8.57k [00:00<00:00, 16.9MB/s]


## 1.1. Importing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# default is 120 seconds
%autosave 60

# showing all columns
pd.set_option('display.max_columns', None)

Autosaving every 60 seconds


In [ ]:
# update the genres for the missing values
train_set.loc[train_set['title'] == 'Unchained Melody', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Someone Elses Roses', 'top genre'] = 'adult standards'
train_set.loc[train_set['title'] == 'Drinks On The House', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Pachuko Hop', 'top genre'] = 'blues'
train_set.loc[train_set['title'] == 'Little Things Means A Lot', 'top genre'] = 'blues'
train_set.loc[train_set['title'] == 'The Lady Is A Tramp', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'If I Give My Heart To You', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Happy Days And Lonely Nights','top genre'] = 'rock'
train_set.loc[train_set['title'] == 'Stairway Of Love','top genre'] = 'rock'
train_set.loc[train_set['title'] == 'You', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'No Other Love' , 'top genre'] = 'adult standards'
train_set.loc[train_set['title'] == "Hot Diggity" , 'top genre'] = 'folk'
train_set.loc[train_set['title'] == "Ain't That Just the Way" , 'top genre'] = 'r&b'
train_set.loc[train_set['title'] == "I Promised Myself" , 'top genre'] = 'pop'
# dropping NULL values  (I've Waited So Long Anthony Newley = ? (Dance/ Electronic))
train_set = train_set.dropna(axis=0)

# resetting index to fill out the dropped row
train_set.reset_index(drop=True, inplace=True)
# check again for null values
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         452 non-null    int64 
 1   title      452 non-null    object
 2   artist     452 non-null    object
 3   year       452 non-null    int64 
 4   bpm        452 non-null    int64 
 5   nrgy       452 non-null    int64 
 6   dnce       452 non-null    int64 
 7   dB         452 non-null    int64 
 8   live       452 non-null    int64 
 9   val        452 non-null    int64 
 10  dur        452 non-null    int64 
 11  acous      452 non-null    int64 
 12  spch       452 non-null    int64 
 13  pop        452 non-null    int64 
 14  top genre  452 non-null    object
dtypes: int64(12), object(3)
memory usage: 53.1+ KB


In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
train_set.rename(columns={'top genre': 'top_genre'}, inplace=True)
train_set.head()

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,28,150,75,3,44,adult standards
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,pop
2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,67,245,11,3,77,adult standards
3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,33,232,25,3,67,adult standards
4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,40,193,45,3,63,glam rock


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

cid = 'd27e82b21add4ca8952dcc2db0297253'
client_secret = '53d0ae49f7044bc390babd37b80e4a21'

# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, artist):
    # Search for the song by name and artist, limit=5 to get top 5 results
    result = sp.search(q=f'track:{name} artist:{artist}', type='track', limit=3)
    items = result['tracks']['items']

    if items:
        return items
    else:
        return []

# List to hold song data
song_data = []

for song_index in range(len(test_set)):
    tracks = search_song(test_set['title'][song_index], test_set['artist'][song_index])
    for track in tracks:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis
df_test = pd.json_normalize(song_data)

# Display or further process the DataFrame as needed
df_test


ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=6E5pe8Zb3Od3ToEBwRxHSt:
 Max Retries, reason: too many 429 error responses

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

cid = 'd27e82b21add4ca8952dcc2db0297253'
client_secret = '53d0ae49f7044bc390babd37b80e4a21'

# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, year, artist):
    # Search for the song by name and year, limit=1 ensures we are getting the most relevant result
    result = sp.search(q=f'track:{name} year:{year} artist:{artist}', type='track', limit=1)
    items = result['tracks']['items']
    if items:
        # Assuming the first track is the one we want
        return items[0]
    else:
        return None

# List to hold song data
song_data = []

train_set
for song in range(len(train_set)):
    track = search_song(train_set['title'][song], train_set['year'][song],train_set['artist'][song])
    if track:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis (optional)
df = pd.json_normalize(song_data)

print(df)


    track_features.album.album_type  \
0                       compilation   
1                       compilation   
2                             album   
3                             album   
4                             album   
..                              ...   
336                     compilation   
337                     compilation   
338                           album   
339                           album   
340                           album   

                          track_features.album.artists  \
0    [{'external_urls': {'spotify': 'https://open.s...   
1    [{'external_urls': {'spotify': 'https://open.s...   
2    [{'external_urls': {'spotify': 'https://open.s...   
3    [{'external_urls': {'spotify': 'https://open.s...   
4    [{'external_urls': {'spotify': 'https://open.s...   
..                                                 ...   
336  [{'external_urls': {'spotify': 'https://open.s...   
337  [{'external_urls': {'spotify': 'https://open.s...   
338  [{'e

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

cid = 'd27e82b21add4ca8952dcc2db0297253'
client_secret = '53d0ae49f7044bc390babd37b80e4a21'

# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, year, artist):
    # Search for the song by name and year, limit=1 ensures we are getting the most relevant result
    result = sp.search(q=f'track:{name} year:{year} artist:{artist}', type='track', limit=1)
    items = result['tracks']['items']
    if items:
        # Assuming the first track is the one we want
        return items[0]
    else:
        return None

# List to hold song data
song_data = []

test_set
for song in range(len(test_set)):
    track = search_song(test_set['title'][song], test_set['year'][song],test_set['artist'][song])
    if track:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis (optional)
df_test = pd.json_normalize(song_data)

print(df_test)


   track_features.album.album_type  \
0                            album   
1                      compilation   
2                            album   
3                            album   
4                      compilation   
..                             ...   
79                           album   
80                           album   
81                           album   
82                           album   
83                           album   

                         track_features.album.artists  \
0   [{'external_urls': {'spotify': 'https://open.s...   
1   [{'external_urls': {'spotify': 'https://open.s...   
2   [{'external_urls': {'spotify': 'https://open.s...   
3   [{'external_urls': {'spotify': 'https://open.s...   
4   [{'external_urls': {'spotify': 'https://open.s...   
..                                                ...   
79  [{'external_urls': {'spotify': 'https://open.s...   
80  [{'external_urls': {'spotify': 'https://open.s...   
81  [{'external_urls': {'spoti

In [ ]:
df.iloc[0:5, 45:].head(5)

,artist_features.followers.total,artist_features.genres,artist_features.href,artist_features.id,artist_features.images,artist_features.name,artist_features.popularity,artist_features.type,artist_features.uri
0,424476,"[adult standards, brill building pop, easy lis...",https://api.spotify.com/v1/artists/3EY5DxGdy7x...,3EY5DxGdy7x4GelivOjS2Q,"[{'height': 1196, 'url': 'https://i.scdn.co/im...",Connie Francis,53,artist,spotify:artist:3EY5DxGdy7x4GelivOjS2Q
1,7481417,"[disco, mellow gold, soft rock]",https://api.spotify.com/v1/artists/1LZEQNv7sE1...,1LZEQNv7sE11VDY3SdxQeN,"[{'height': 733, 'url': 'https://i.scdn.co/ima...",Bee Gees,77,artist,spotify:artist:1LZEQNv7sE11VDY3SdxQeN
2,1449771,"[adult standards, operatic pop, soft rock]",https://api.spotify.com/v1/artists/7jmTilWYlKO...,7jmTilWYlKOuavFfmQAcu6,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Barbra Streisand,63,artist,spotify:artist:7jmTilWYlKOuavFfmQAcu6
3,12803780,"[glam rock, mellow gold, piano rock, rock]",https://api.spotify.com/v1/artists/3PhoLpVuITZ...,3PhoLpVuITZKcymswpck5b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Elton John,83,artist,spotify:artist:3PhoLpVuITZKcymswpck5b
4,53205850,"[dance pop, pop]",https://api.spotify.com/v1/artists/0du5cEVh5yT...,0du5cEVh5yTK9QJze8zA0C,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Bruno Mars,88,artist,spotify:artist:0du5cEVh5yTK9QJze8zA0C


In [ ]:
retained_cols = [
#'track_features.album.album_type',
# 'track_features.album.artists',
 'track_features.album.available_markets',
# 'track_features.album.external_urls.spotify',
# 'track_features.album.href',
# 'track_features.album.id',
# 'track_features.album.images',
 'track_features.album.name',
 'track_features.album.release_date',
# 'track_features.album.release_date_precision',
 'track_features.album.total_tracks',
# 'track_features.album.type',
# 'track_features.album.uri',
 #'track_features.artists',
 'track_features.available_markets',
 'track_features.disc_number',
 'track_features.duration_ms',
 'track_features.explicit',
# 'track_features.external_ids.isrc',
# 'track_features.external_urls.spotify',
# 'track_features.href',
# 'track_features.id',
# 'track_features.is_local',
 'track_features.name',
 'track_features.popularity',
# 'track_features.preview_url',
 'track_features.track_number',
# 'track_features.type',
# 'track_features.uri',
 'track_features.danceability',
 'track_features.energy',
 'track_features.key',
 'track_features.loudness',
 'track_features.mode',
 'track_features.speechiness',
 'track_features.acousticness',
 'track_features.instrumentalness',
 'track_features.liveness',
 'track_features.valence',
 'track_features.tempo',
# 'track_features.track_href',
# 'track_features.analysis_url',
 'track_features.time_signature',
# 'artist_features.external_urls.spotify',
# 'artist_features.followers.href',
 'artist_features.followers.total',
 'artist_features.genres',
# 'artist_features.href',
# 'artist_features.id',
# 'artist_features.images',
 'artist_features.name',
 'artist_features.popularity',
# 'artist_features.type',
# 'artist_features.uri'
  ]

In [ ]:
df[retained_cols]

,track_features.album.available_markets,track_features.album.name,track_features.album.release_date,track_features.album.total_tracks,track_features.available_markets,track_features.disc_number,track_features.duration_ms,track_features.explicit,track_features.is_local,track_features.name,...,track_features.acousticness,track_features.instrumentalness,track_features.liveness,track_features.valence,track_features.tempo,track_features.time_signature,artist_features.followers.total,artist_features.genres,artist_features.name,artist_features.popularity
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Her Greatest Hits & Finest Performances,1996-01-01,61,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,150573,False,False,My Happiness,...,0.824,0.000002,0.1150,0.332,108.765,3,424476,"[adult standards, brill building pop, easy lis...",Connie Francis,53
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Greatest,1979-01-01,27,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,245200,False,False,How Deep Is Your Love,...,0.105,0.000000,0.1330,0.674,104.938,4,7481417,"[disco, mellow gold, soft rock]",Bee Gees,77
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Guilty,1980-09-23,9,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,231840,False,False,Woman in Love,...,0.248,0.000126,0.1330,0.331,169.735,4,1449771,"[adult standards, operatic pop, soft rock]",Barbra Streisand,63
3,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Goodbye Yellow Brick Road (Remastered),1973-10-05,17,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,192827,False,False,Goodbye Yellow Brick Road - Remastered 2014,...,0.446,0.001410,0.1540,0.397,121.227,4,12803780,"[glam rock, mellow gold, piano rock, rock]",Elton John,83
4,"[AR, AU, BN, BO, BZ, CA, CL, CO, CR, DO, EC, G...",Doo-Wops & Hooligans,2010-10-05,10,"[AR, AU, BN, BO, BZ, CA, CL, CO, CR, DO, EC, G...",1,222091,False,False,Grenade,...,0.148,0.000000,0.1070,0.245,110.444,4,53205850,"[dance pop, pop]",Bruno Mars,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,"[AR, AU, AT, BE, BO, BR, BG, CL, CO, CR, CY, C...",The Very Best of Frankie Valli & The 4 Seasons,2003-01-14,20,"[AR, AU, AT, BE, BO, BR, BG, CL, CO, CR, CY, C...",1,181560,False,False,Rag Doll,...,0.755,0.000088,0.0665,0.611,120.935,4,613249,"[adult standards, bubblegum pop, doo-wop, loun...",Frankie Valli & The Four Seasons,62
337,[US],Play Soft Rock,2017-03-24,25,[US],1,160880,False,False,A World Without Love,...,0.232,0.000000,0.1080,0.752,129.861,4,22920,"[beatlesque, british blues, british invasion]",Peter And Gordon,38
338,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Ella Fitzgerald Sings The George And Ira Gersh...,1959-01-01,59,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,214253,False,False,But Not For Me,...,0.920,0.004490,0.1020,0.162,80.141,4,2407394,"[adult standards, jazz, jazz blues, soul, swin...",Ella Fitzgerald,70
339,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",Surf City And Other Swingin' Cities,2010-01-01,12,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,147267,False,False,Surf City,...,0.497,0.000050,0.2320,0.964,147.843,4,110806,"[bubblegum pop, classic garage rock, doo-wop, ...",Jan & Dean,38


In [ ]:
print(df['track_features.is_local'].unique())

[False]


In [ ]:
train_set[train_set.title df['track_features.name']]

0                                     My Happiness
1                            How Deep Is Your Love
2                                    Woman in Love
3      Goodbye Yellow Brick Road - Remastered 2014
4                                          Grenade
                          ...                     
336                                       Rag Doll
337                           A World Without Love
338                                 But Not For Me
339                                      Surf City
340                                        Dilemma
Name: track_features.name, Length: 341, dtype: object

In [ ]:
filtered_train_set = train_set[~train_set['title'].isin(df['track_features.name'])]
filtered_train_set

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,pop
8,9,Someone Elses Roses,Joan Regan,2019,100,14,29,-10,15,32,143,89,3,34,adult standards
9,10,You Belong to My Heart,Timi Yuro,2013,108,39,46,-10,16,50,126,82,3,35,adult standards
12,13,Laugh Laugh,The Beau Brummels,2006,129,68,49,-9,10,75,177,30,5,30,brill building pop
13,14,What Do You Want,Adam Faith,1989,147,37,55,-9,24,85,98,84,4,35,brill building pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,440,April Love,Pat Boone,1962,91,17,39,-15,8,50,161,83,3,48,adult standards
439,441,Womanizer,Britney Spears,2009,139,70,72,-5,9,24,224,7,6,74,dance pop
443,445,Needles and Pins - Mono,The Searchers,1964,124,95,56,-7,6,89,131,5,4,56,adult standards
450,452,It's Gonna Be Me,*NSYNC,2000,165,87,64,-5,6,88,191,5,8,62,boy band


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

cid = '75b8b1605cb64f42bf5705c1e9388076'
client_secret = 'd77064f8876340fa96592c7d001599fa'

# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, artist):
    # Search for the song by name and year, limit=1 ensures we are getting the most relevant result
    result = sp.search(q=f'track:{name} artist:{artist}', type='track', limit=1)
    items = result['tracks']['items']
    if items:
        # Assuming the first track is the one we want
        return items[0]
    else:
        return None

# List to hold song data
song_data = []

test_set
for song in range(len(test_set)):
    track = search_song(test_set['title'][song],test_set['artist'][song])
    if track:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis (optional)
df_test = pd.json_normalize(song_data)

print(df_test)


ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=6E5pe8Zb3Od3ToEBwRxHSt:
 Max Retries, reason: too many 429 error responses

In [ ]:
path = 'train_enrich.csv'
df.to_csv(path)
path = 'test_enrich.csv'
df_test.to_csv(path)

## 2.1. Remove duplicate

In [ ]:
train_set = train_set.drop_duplicates()
len(train_set)

453

## 2.2. Feature enginerring

In [ ]:
# With the year range so long, ranging from 1948 to 2019, we can group the years into group of 5:

def assign_group(year):
    if 1948 <= year <= 1954:
        return 1
    else:
        return ((year - 1954)//5)+2

# Apply the calculation
train_set['year'] = train_set['year'].apply(assign_group)
test_set['year'] = test_set['year'].apply(assign_group)

In [ ]:
y_train = train_set['top_genre']
X_train = train_set.drop(['top_genre'], axis=1)
X_test = test_set.copy()

In [ ]:
X_train

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,My Happiness,Connie Francis,10,107,31,45,-8,1.989560,28,6.806697,1.235841,0.580183,44
1,2,Unchained Melody,The Teddy Bears,13,114,44,53,-8,1.989560,47,6.671941,0.742877,0.580183,37
2,3,How Deep Is Your Love,Bee Gees,7,105,36,63,-9,1.989560,67,7.705224,-0.533518,0.580183,77
3,4,Woman in Love,Barbra Streisand,7,170,28,47,-16,1.989560,33,7.602735,0.088752,0.580183,67
4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,5,121,47,56,-8,2.062160,40,7.261670,0.652019,0.580183,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,449,But Not For Me,Ella Fitzgerald,3,80,22,18,-17,1.852779,16,7.452144,1.497015,0.603482,45
449,450,Surf City,Jan & Dean,13,148,81,53,-13,2.269691,96,6.770829,0.764797,0.580183,50
450,451,Dilemma,Nelly,11,168,55,73,-8,2.203427,61,8.019814,0.017299,0.650526,77
451,452,It's Gonna Be Me,*NSYNC,11,165,87,64,-5,1.575528,88,7.242591,-0.979164,0.637675,62


## 3.1. Features scaling for numeric features

In [ ]:
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox

yeo_johnson_transformer = PowerTransformer(method='yeo-johnson')
X_train['acous'] = yeo_johnson_transformer.fit_transform(X_train[['acous']])
X_test['acous'] = yeo_johnson_transformer.transform(X_test[['acous']])
for col in ['spch', 'live', 'dur']:
    X_train[col], lambdas = boxcox(X_train[col]+1)
    X_test[col] = boxcox(X_test[col] +1, lmbda=lambdas)

In [ ]:
# Scale the dataset using robust scaler to mitigate the effect of outliers
standard_scaler = StandardScaler()

# Scaling the dataset while keeping the labels intact
columns_name = X_train.columns

# Scaling will return a ndarray so we will need to transform it back to a dataframe
X_train_scaled = standard_scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns= columns_name)

# Scaling the X_test
X_test_scaled = standard_scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns= columns_name)

ValueError: could not convert string to float: 'My Happiness'

In [ ]:
cid = 'd27e82b21add4ca8952dcc2db0297253'
client_secret = '53d0ae49f7044bc390babd37b80e4a21'

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Initialize Spotipy with user credentails
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid,client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, year):
    # Search for the song by name and year, limit=1 ensures we are getting the most relevant result
    result = sp.search(q=f'track:{name} year:{year}', type='track', limit=1)
    items = result['tracks']['items']
    if items:
        # Assuming the first track is the one we want
        return items[0]
    else:
        return None

# List to hold song data
song_data = []

songs = [
    {"name": "bad guy","artist":"Billie Eilish", "year": 2019},
    {"name": "7 rings", "artist":"Ariana Grande","year": 2019},
]

for song in songs:
    track = search_song(song['name'], song['year'])
    if track:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis (optional)
df = pd.DataFrame(song_data)
print(df)

TypeError: search_song() takes 2 positional arguments but 3 were given

In [ ]:
df.columns

Index(['track_features.album.album_type', 'track_features.album.artists',
       'track_features.album.available_markets',
       'track_features.album.external_urls.spotify',
       'track_features.album.href', 'track_features.album.id',
       'track_features.album.images', 'track_features.album.name',
       'track_features.album.release_date',
       'track_features.album.release_date_precision',
       'track_features.album.total_tracks', 'track_features.album.type',
       'track_features.album.uri', 'track_features.artists',
       'track_features.available_markets', 'track_features.disc_number',
       'track_features.duration_ms', 'track_features.explicit',
       'track_features.external_ids.isrc',
       'track_features.external_urls.spotify', 'track_features.href',
       'track_features.id', 'track_features.is_local', 'track_features.name',
       'track_features.popularity', 'track_features.preview_url',
       'track_features.track_number', 'track_features.type',
       

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Search for songs
results = sp.search(q='track:bad guy artist:Billie Eilish', type='track')

# Extract metadata
for track in results['tracks']['items']:
    print('Track Name:', track['name'])
    print('Artist:', track['artists'][0]['name'])
    print('Album:', track['album']['name'])
    print('Release Date:', track['album']['release_date'])
    print('Popularity:', track['popularity'])
    print('--------------------------')


Track Name: bad guy
Artist: Billie Eilish
Album: WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?
Release Date: 2019-03-29
Popularity: 85
--------------------------
Track Name: bad guy (with Justin Bieber)
Artist: Billie Eilish
Album: bad guy (with Justin Bieber)
Release Date: 2019-07-11
Popularity: 68
--------------------------
Track Name: bad guy
Artist: Billie Eilish
Album: Playlist for Gamers
Release Date: 2021-08-18
Popularity: 57
--------------------------
Track Name: bad guy
Artist: Billie Eilish
Album: Angel's Most Played Tracks
Release Date: 2021-08-24
Popularity: 38
--------------------------
Track Name: bad guy
Artist: Billie Eilish
Album: Model Walk Songs
Release Date: 2021-08-20
Popularity: 39
--------------------------
Track Name: bad guy
Artist: Billie Eilish
Album: NOW That's What I Call A Decade! 2010's
Release Date: 2020-08-07
Popularity: 22
--------------------------
Track Name: bad guy
Artist: Billie Eilish
Album: NOW That's What I Call Music, Vol. 71
Release Date: 2019-08-

## 3.2. Encoding

## 4.0. Importing cleaned testing result

In [ ]:
y_test = pd.read_csv('y_true.csv',header = None, names=['top_genre'])
y_test

,top_genre
0,dance pop
1,glam rock
2,glam rock
3,big beat
4,appalachian folk
...,...
108,east coast hip hop
109,bubblegum dance
110,east coast hip hop
111,disco


### 3.5.3. Weighted Loss function

In [ ]:
# Create and train the Random Forest model with class weights
RFC_model = RandomForestClassifier(n_estimators=400, max_depth=17, random_state=42,
                                  class_weight='balanced')

# Fit the model
RFC_model.fit(X_train_full, y_train_full)

In [ ]:
rfc_pred = RFC_model.predict(X_test_full)
rfc_pred = label_encoder1.inverse_transform(rfc_pred)
print(accuracy_score(y_test_true, rfc_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_true, rfc_pred))

In [ ]:
rfc_importances = RFC_model.feature_importances_
rfc_importances

In [ ]:
train_set = train_set.drop_duplicates()
len(train_set)

451